# Lista de Exercícios 3: Aproximação de Funções

#### Disciplina: Aprendizado por Reforço
#### Professor: Luiz Chaimowicz
#### Monitores: Marcelo Lemos e Ronaldo Vieira

---

## Instruções

- ***SUBMISSÕES QUE NÃO SEGUIREM AS INSTRUÇÕES A SEGUIR NÃO SERÃO AVALIADAS.***
- Leia atentamente toda a lista de exercícios e familiarize-se com o código fornecido antes de começar a implementação.
- Os locais onde você deverá escrever suas soluções estão demarcados com comentários `# YOUR CODE HERE` ou `YOUR ANSWER HERE`.
- **Não altere o código fora das áreas indicadas, nem adicione ou remova células. O nome deste arquivo também não deve ser modificado.**
- Antes de submeter, certifique-se de que o código esteja funcionando do início ao fim sem erros.
- Submeta apenas este notebook (*ps3.ipynb*) com as suas soluções no Moodle.
- Prazo de entrega: 30/10/2025. Submissões fora do prazo terão uma penalização de -20% da nota final por dia de atraso.
- Utilize a [documentação do Gymnasium](https://gymnasium.farama.org/) para auxiliar sua implementação.
- Em caso de dúvidas entre em contato pelo fórum "Dúvidas com relação aos exercícios e trabalho de curso" no moodle da Disciplina.

---

## Mountain Car

Mountain Car é um ambiente no qual um agente precisa conduzir um carro até o topo de uma montanha. No entanto, o motor do carro não é potente o suficiente para subir diretamente até o topo. Por isso, o agente precisa aprender a aproveitar o impulso - movendo-se para frente e para trás - para ganhar velocidade suficiente e alcançar o topo.

![](https://gymnasium.farama.org/_images/mountain_car.gif)

O espaço de observação é contínuo e composto por dois valores: a posição e a velocidade do carro. O agente pode escolher entre três ações discretas: acelerar o carro para a esquerda, acelerar para a direita, ou não acelerar. A posição inicial do agente é definida de forma uniformemente aleatória no intervalo $[-0.6, -0.4]$. O episódio se encerra quando o carro atinge o topo da montanha à direita (posição $0.5$) ou quando o limite de 200 passos é atingido. A cada passo, o agente recebe uma penalidade de $-1$, incentivando-o a alcançar o objetivo no menor número possível de passos. Para mais detalhes sobre o ambiente leia a [documentação do gymnasium](https://gymnasium.farama.org/environments/classic_control/mountain_car/).

Devido à natureza contínua do espaço de estados, métodos tabulares não são eficazes no Mountain Car. Assim, é comum o uso de técnicas de aproximação de função para solucionar o problema de forma eficiente. Nesta lista de exercícios, você irá trabalhar com o ambiente descrito acima. Seu objetivo será implementar o algoritmo *Semi-Gradient Episodic Sarsa* com aproximação linear e explorar como diferentes técnicas de construção de features influenciam o desempenho do agente.

## Semi-Gradient Episodic Sarsa

Nesta atividade, você implementará um agente baseado no algoritmo Semi-Gradient Episodic SARSA, utilizando uma **função de valor linear**.

Antes de iniciar sua implementação, analise a interface `FeatureExtractor` fornecida abaixo. Ela será a base para os construtores de features que você implementará na próxima seção. O agente utilizará um objeto com essa interface para extrair features a partir das observações dos estados.

In [ ]:
from abc import ABC, abstractmethod

import numpy as np
import gymnasium as gym
import sklearn
import matplotlib.pyplot as plt

In [ ]:
class FeatureExtractor(ABC):
    """
    Interface for feature extractors that convert environment states into feature vectors.
    """

    @property
    @abstractmethod
    def feature_size(self):
        """
        Property that returns the size of the feature vector produced by this extractor.

        Returns:
            An integer representing the length of the feature vector.
        """
        pass

    @abstractmethod
    def extract(self, state):
        """
        Converts a raw state into a 1D feature vector.

        Args:
            state: The observation state from the environment.

        Returns:
            The extracted feature vector representation.
        """
        pass

Siga as instruções abaixo para implementar seu agente:

1. Implemente o método `__init__` que inicializa um novo agente. Ele deve receber como parâmetros o espaço de observações, o espaço de ações, um construtor de features (`feature_extractor`), a taxa de aprendizado $\alpha$, o fator de desconto $\gamma$, e o parâmetro de exploração $\varepsilon$. Inicialize os pesos do modelo na variável `self.weights`.
2. Implemente o método `compute_q_values`, que recebe um vetor de features e calcula os *Q-values* de acordo com a entrada.
3. Implemente o método `choose_action`, responsável por escolher uma ação a partir de um estado observado, seguindo a política $\varepsilon$-greedy.
4. Implemente o método `learn`, que atualiza os pesos do agente com base na experiência obtida durante a interação com o ambiente.
5. Implemente o método `train`, que executa o loop de treinamento do algoritmo Sarsa. O ambiente de treinamento e o número de episódios devem ser fornecidos como parâmetros de entrada. O método deve retornar uma lista com a soma das recompensas obtidas ao longo de cada episódio.

In [ ]:
class SGESAgent:
    def __init__(self, observation_space, action_space, feature_extractor, alpha, gamma, epsilon):
        """
        Initializes the Semi-Gradient Episodic Sarsa (SGES) Agent.

        Args:
            observation_space: The environment's observation space.
            action_space: The environment's action space.
            feature_extractor (FeatureExtractor): An object to convert states to feature vectors.
            [cite_start]alpha (float): The step-size parameter (learning rate). [cite: 31, 795]
            gamma (float): The discount-rate parameter.
            [cite_start]epsilon (float): The probability for exploration (epsilon-greedy). [cite: 31, 795]
        """
        # Store essential parameters
        self.action_space = action_space
        self.n_actions = action_space.n
        self.feature_extractor = feature_extractor
        self.feature_size = feature_extractor.feature_size
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

        # [cite_start]Initialize value-function weights (theta) [cite: 31, 796]
        # We use a linear function approximator: q(s, a, w) = x(s)^T * w_a
        # Therefore, our weights 'w' will be a matrix of shape (feature_size, n_actions),
        # where each column represents the weights for a specific action.
        # [cite_start]We initialize them to zero as suggested by the pseudocode[cite: 31, 796].
        self.weights = np.zeros((self.feature_size, self.n_actions))

        # Initialize a random number generator for reproducibility
        self.rng = np.random.default_rng()

    def compute_q_values(self, features):
        """
        Computes the Q-values for all actions given a state's feature vector.
        [cite_start]This implements the linear approximation q(s, a, w) = x(s)^T * w_a for all 'a'. [cite: 739, 91]

        Args:
            features (np.ndarray): The feature vector x(s) for the current state.

        Returns:
            np.ndarray: A 1D array of Q-values for all possible actions.
        """
        # Perform a matrix multiplication: x(s)^T * W
        # features shape: (feature_size,)
        # self.weights shape: (feature_size, n_actions)
        # Resulting shape: (n_actions,)
        return features @ self.weights

    def choose_action(self, state):
        """
        [cite_start]Chooses an action from a given state using an epsilon-greedy policy. [cite: 38, 803]

        Args:
            state: The current environment state.

        Returns:
            int: The chosen action.
        """
        # Epsilon-greedy exploration
        if self.rng.random() < self.epsilon:
            # Choose a random action
            return self.action_space.sample()
        else:
            # Choose the greedy action
            # 1. Extract features from the state
            features = self.feature_extractor.extract(state)
            # 2. Compute Q-values for all actions
            q_values = self.compute_q_values(features)
            # [cite_start]3. Select the action with the highest Q-value [cite: 27]
            return np.argmax(q_values)

    def learn(self, state, action, reward, next_state, next_action, done):
        """
        [cite_start]Updates the agent's weights using the semi-gradient Sarsa update rule. [cite: 38, 803]

        Args:
            state: The state S.
            action: The action A.
            reward: The reward R.
            next_state: The next state S'.
            next_action: The next action A'.
            done (bool): True if S' is a terminal state, False otherwise.
        """
        # 1. Get the feature vector for the current state, x(S)
        features = self.feature_extractor.extract(state)

        # 2. Get the predicted Q-value for the current state-action pair: q(S, A, w)
        q_value = self.compute_q_values(features)[action]

        # 3. Determine the update target (U_t)
        if done:
            # [cite_start]If S' is terminal, the target is just the reward R. [cite: 35, 36, 800, 801]
            q_target = reward
        else:
            # [cite_start]If S' is not terminal, the target is R + gamma * q(S', A', w) [cite: 38, 803]

            # 3a. Get the feature vector for the next state, x(S')
            next_features = self.feature_extractor.extract(next_state)

            # 3b. Get the Q-value for the *next* state-action pair: q(S', A', w)
            next_q_value = self.compute_q_values(next_features)[next_action]

            # 3c. Calculate the full Sarsa target
            q_target = reward + self.gamma * next_q_value

        # 4. Calculate the TD error (delta)
        # delta = [Target - Prediction]
        td_error = q_target - q_value

        # 5. Get the gradient of the value function approximator, grad(q(S, A, w))
        # For a linear function q(S, A, w) = x(S)^T * w_A,
        # [cite_start]the gradient with respect to w_A is just the feature vector x(S). [cite: 91, 739]
        gradient = features

        # 6. Perform the semi-gradient update
        # [cite_start]w_A <- w_A + alpha * delta * grad(q(S, A, w)) [cite: 38, 803]
        # We only update the weights for the action that was actually taken.
        self.weights[:, action] += self.alpha * td_error * gradient

    def train(self, env, episodes):
        """
        Runs the main training loop for the SGESAgent.

        Args:
            env: The Gymnasium environment to train in.
            episodes (int): The total number of episodes to run.

        Returns:
            list: A list containing the total return (sum of rewards) for each episode.
        """
        # List to store the total reward for each episode
        episode_returns = []

        # [cite_start]Loop for each episode [cite: 32, 797]
        for episode in range(episodes):
            total_reward = 0

            # [cite_start]Initialize S (state) [cite: 33, 798]
            state, info = env.reset()

            # [cite_start]Choose A (action) from S using the policy [cite: 33, 798]
            action = self.choose_action(state)

            done = False

            # [cite_start]Loop for each step of the episode [cite: 33, 798]
            while not done:
                # [cite_start]Take action A, observe R, S' [cite: 34, 799]
                next_state, reward, terminated, truncated, info = env.step(action)
                done = terminated or truncated
                total_reward += reward

                # [cite_start]Choose A' (next action) from S' using the policy [cite: 38, 803]
                next_action = self.choose_action(next_state)

                # Perform the Sarsa update (S, A, R, S', A')
                # [cite_start]The 'done' flag is passed to handle the terminal update case [cite: 35, 36, 800, 801]
                self.learn(state, action, reward, next_state, next_action, done)

                # S <- S'; [cite_start]A <- A' [cite: 39, 40, 804]
                state = next_state
                action = next_action

            # Store the total return for this episode
            episode_returns.append(total_reward)

        return episode_returns

In [ ]:
# Não altere ou remova esta célula

In [ ]:
# Não altere ou remova esta célula

In [ ]:
# Não altere ou remova esta célula

In [ ]:
# Não altere ou remova esta célula

## Construção de Features

Com o algoritmo do agente pronto, você deverá implementar 3 modelos de construtores de features diferentes:

1. **Identidade**
2. **Tile Coding**
3. **Funções de Base Radial**

### Identidade

6. Implemente o `IdentityFeatureExtractor`, cujo método `extract` apenas retorna a observação de entrada, sem realizar nenhuma operação nela.

In [ ]:
class IdentityFeatureExtractor(FeatureExtractor):
    def __init__(self, observation_space):
        """
        Initializes the IdentityFeatureExtractor.

        Args:
            observation_space: The environment's observation space.
        """
        # Store the observation space to determine the feature size.
        self._observation_space = observation_space

        # The feature size is the dimensionality of the observation space.
        self._feature_size = observation_space.shape[0]

    @property
    def feature_size(self):
        """
        Property that returns the size of the feature vector.
        For identity, this is the same as the observation space dimension.

        Returns:
            An integer representing the length of the feature vector.
        """
        return self._feature_size

    def extract(self, state):
        """
        Converts a raw state into a 1D feature vector.
        As per the instructions, this extractor performs no transformation.

        Args:
            state: The observation state from the environment.

        Returns:
            The untransformed state as the feature vector.
        """
        # Returns the state as-is, performing no operations.
        return state

In [ ]:
# Não altere ou remova esta célula

7. Treine um novo agente no ambiente Mountain Car por 200 episódios utilizando o `IdentityFeatureExtractor`. O agente deve ser treinado com os seguintes parâmetros: taxa de aprendizado $\alpha = 0.01$, fator de desconto $\gamma = 0.99$ e parâmetro de exploração $\varepsilon = 0.1$. Armazene os retornos episódicos, obtidos no método `train`, na variável `identity_agent`.

**Nota:** Não se preocupe se o desempenho do agente com o `IdentityFeatureExtractor` for insatisfatório. Ele será utilizado apenas como baseline, permitindo observar o comportamento do agente quando nenhuma transformação é aplicada às observações do ambiente.

In [ ]:
env = gym.make("MountainCar-v0")

# YOUR CODE HERE
# Define the hyperparameters specified in Task 7
ALPHA = 0.01
GAMMA = 0.99
EPSILON = 0.1
EPISODES = 200

# 1. Instantiate the IdentityFeatureExtractor
# This extractor will just pass the (position, velocity) state as features.
identity_feat_extractor = IdentityFeatureExtractor(env.observation_space)

# 2. Instantiate the SGESAgent
# We pass the environment, the feature extractor, and the hyperparameters.
agent = SGESAgent(
    observation_space=env.observation_space,
    action_space=env.action_space,
    feature_extractor=identity_feat_extractor,
    alpha=ALPHA,
    gamma=GAMMA,
    epsilon=EPSILON
)

# 3. Train the agent for 200 episodes
# [cite_start]The `train` method executes the episodic loop from the pseudocode [cite: 78, 797]
# and returns the list of total rewards for each episode.
identity_agent = agent.train(env, EPISODES)

env.close()

mean_return = np.mean(identity_agent)
print(f"Mean Return: {mean_return:.2f}")

last_mean = np.mean(identity_agent[-20:])
print(f"Mean Return of the Last 20 Episodes: {last_mean:.2f}")

In [ ]:
# Não altere ou remova esta célula

### Tile Coding

8. Implemente o `TileCodingFeatureExtractor`, cujo método `extract` gera um vetor de features utilizando a técnica de *tile coding*. A quantidade de tilings a ser gerada é definida pelo parâmetro `num_tilings`, enquanto a quantidade de divisões por dimensão em cada tiling é definida pelo parâmetro `tiles_per_dim`. Ambos valores são fornecidos ao construtor da classe. Por exemplo: um `TileCodingFeatureExtractor` com `num_tilings = 3` e `tiles_per_dim = 2` deve gerar 3 tilings, cada um dividindo o espaço em uma grade de tamanho 2$\times$2.

In [ ]:
class TileCodingFeatureExtractor(FeatureExtractor):
    def __init__(self, observation_space, num_tilings, tiles_per_dim):
        """
        Initializes the TileCodingFeatureExtractor.

        This implementation creates `num_tilings` grids (tilings),
        each of size `tiles_per_dim` x `tiles_per_dim`.
        The grids are offset from each other to provide a finer-grained
        and generalized feature representation.

        Args:
            observation_space: The environment's observation space.
            num_tilings (int): The number of tilings to create.
            tiles_per_dim (int): The number of tiles for each dimension
                                (e.g., 8 for an 8x8 grid).
        """
        # Store state space boundaries
        self.low = observation_space.low
        self.high = observation_space.high

        # Store tiling parameters
        self.num_tilings = num_tilings
        self.tiles_per_dim = tiles_per_dim
        self.num_dims = len(self.low)

        # Calculate the width of a single tile in each dimension
        # This matches the method from the provided Lisp code,
        # [cite_start]e.g., pos-width = (range / tiles_per_dim) [cite: 40, 41]
        self.tile_widths = (self.high - self.low) / self.tiles_per_dim

        # Calculate the total size of one tiling's grid (e.g., 8x8 = 64)
        self.tiling_size = self.tiles_per_dim ** self.num_dims

        # Calculate the total number of features
        # This is (number of tilings) * (features per tiling)
        self._feature_size = self.num_tilings * self.tiling_size

        # Create offsets for each tiling
        # We will offset each tiling by a fraction of a tile width
        # This creates the staggered, overlapping grids.
        self.tiling_offsets = np.zeros((self.num_tilings, self.num_dims))
        for i in range(self.num_tilings):
            # Asymmetrical offset: offset dimension 'j' by 'i * (2j + 1)'
            # This is a common and effective method.
            # We divide by num_tilings to scale the offset.
            for j in range(self.num_dims):
                self.tiling_offsets[i, j] = (i * (2 * j + 1) / self.num_tilings) * self.tile_widths[j]

        # Pre-calculate multipliers for flattening the N-D tile coordinates
        # For 2D: [tiles_per_dim^0, tiles_per_dim^1] = [1, 8] (if 8x8)
        # Index = (coord[0] * 1) + (coord[1] * 8)
        self.index_multipliers = self.tiles_per_dim ** np.arange(self.num_dims)

    @property
    def feature_size(self):
        """
        Property that returns the total size of the feature vector.

        Returns:
            An integer representing the total number of features.
        """
        return self._feature_size

    def extract(self, state):
        """
        Converts a raw state into a sparse 1D binary feature vector.
        The vector will have exactly `num_tilings` active features (1.0),
        one for each tiling.

        Args:
            state: The observation state (position, velocity).

        Returns:
            The extracted binary feature vector.
        """
        # Create a sparse vector (all zeros)
        features = np.zeros(self._feature_size)

        # Loop over each of the 'num_tilings'
        for i in range(self.num_tilings):
            # Apply the offset for the current tiling
            offset_state = state + self.tiling_offsets[i]

            # Find the discrete (x, y) coordinates in this tiling's grid
            coordinates = np.floor((offset_state - self.low) / self.tile_widths).astype(int)

            # Clip coordinates to be within the grid boundaries [0, tiles_per_dim - 1]
            coordinates = np.clip(coordinates, 0, self.tiles_per_dim - 1)

            # Convert the N-D coordinates to a single 1D index for this tiling
            # E.g., for 8x8: (pos_idx * 1) + (vel_idx * 8)
            local_index = np.dot(coordinates, self.index_multipliers)

            # Calculate the global index in the final feature vector
            # This places the local index in the correct 'block' for this tiling
            global_index = (i * self.tiling_size) + local_index

            # Activate the corresponding feature
            features[global_index] = 1.0

        return features

In [ ]:
# Não altere ou remova esta célula

9. Treine um novo agente no ambiente Mountain Car por 200 episódios utilizando o `TileCodingFeatureExtractor`. O agente deve ser treinado com os seguintes parâmetros: taxa de aprendizado $\alpha = 0.01$, fator de desconto $\gamma = 0.99$ e parâmetro de exploração $\varepsilon = 0.1$. Para os parâmetros `num_tilings` e `tiles_per_dim`, utilize os valores que proporcionarem os melhores resultados. Armazene os retornos episódicos, obtidos no método `train`, na variável `tile_agent`.

In [ ]:
env = gym.make("MountainCar-v0")

# YOUR CODE HERE
# --- Hyperparameters ---
# Agent parameters are fixed by the notebook instructions (Task 9)
ALPHA = 0.01
GAMMA = 0.99
EPSILON = 0.1
EPISODES = 200

# Feature Extractor parameters are flexible.
# We will use the values from the reference materials which are
# shown to work well for this specific problem.
#
# The textbook (Chapter 10, Example 10.1) states:
# "We used 8 tilings, with each tile covering 1/8th of the bounded
# [cite_start]distance in each dimension"[cite: 679].
# [cite_start]The Lisp code used to generate the figures also uses 8 tilings[cite: 1017].
NUM_TILINGS = 8
TILES_PER_DIM = 8

# 1. Instantiate the TileCodingFeatureExtractor
tile_feat_extractor = TileCodingFeatureExtractor(
    observation_space=env.observation_space,
    num_tilings=NUM_TILINGS,
    tiles_per_dim=TILES_PER_DIM
)

# 2. Instantiate the SGESAgent
agent = SGESAgent(
    observation_space=env.observation_space,
    action_space=env.action_space,
    feature_extractor=tile_feat_extractor,
    alpha=ALPHA,
    gamma=GAMMA,
    epsilon=EPSILON
)

# 3. Train the agent for 200 episodes
# The `train` method will execute the Sarsa loop and return the
# list of total rewards per episode.
tile_agent = agent.train(env, EPISODES)

env.close()

mean_return = np.mean(tile_agent)
print(f"Mean Return: {mean_return:.2f}")

last_mean = np.mean(tile_agent[-20:])
print(f"Mean Return of the Last 20 Episodes: {last_mean:.2f}")

In [ ]:
# Não altere ou remova esta célula

### Funções de Base Radial

10. Implemente o `RBFFeatureExtractor`, cujo método `extract` gera um conjunto de features baseadas em *Radial Basis Functions*. A quantidade total de componentes a ser gerada é definida pelo parâmetro `n_components`, fornecido ao construtor da classe. Utilize a biblioteca *scikit-learn* (sklearn) para auxiliar sua implementação.

**Importante:** Você pode combinar RBFs com diferentes parâmetros para capturar melhor dinâmicas complexas do ambiente e potencialmente melhorar o desempenho do agente. Experimente diferentes configurações para identificar as combinações que produzem os melhores resultados de aprendizado. Normalizar o vetor de entrada também pode facilitar o aprendizado.

In [ ]:
# We need these sklearn components as suggested by the notebook
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_approximation import RBFSampler

class RBFFeatureExtractor(FeatureExtractor):
    def __init__(self, observation_space, n_components):
        """
        Initializes the RBFFeatureExtractor.

        This implementation uses sklearn's RBFSampler, also known as
        "Random Kitchen Sinks", to approximate an RBF kernel feature map.

        Following the notebook's hint, we:
        1. Normalize the input state using StandardScaler.
        2. Combine multiple RBF samplers with different 'gamma' values
           to capture features at different scales.

        Args:
            observation_space: The environment's observation space.
            n_components (int): The target number of features.
        """
        # --- 1. Fit a StandardScaler to normalize inputs ---
        # We sample from the observation space to get a representative
        # distribution of states for fitting the scaler.
        samples = np.array(
            [observation_space.sample() for _ in range(10000)],
            dtype=np.float32
        )
        self.scaler = StandardScaler()
        self.scaler.fit(samples)

        # --- 2. Create multiple RBF Samplers ---
        # We use a few different gamma values to capture features at
        # different scales, as hinted in the notebook.
        self.gammas = [0.1, 0.5, 1.0, 5.0]
        n_gammas = len(self.gammas)

        # We divide the total components among the samplers.
        # Using ceil ensures we get at least n_components.
        self.n_components_per_sampler = int(
            np.ceil(n_components / n_gammas)
        )

        # The true feature size is the sum of components from all samplers
        self._feature_size = self.n_components_per_sampler * n_gammas

        self.samplers = []
        for gamma in self.gammas:
            # We use a fixed random_state for reproducibility
            sampler = RBFSampler(
                gamma=gamma,
                n_components=self.n_components_per_sampler,
                random_state=42
            )
            # Fit the sampler to the scaled data
            sampler.fit(self.scaler.transform(samples))
            self.samplers.append(sampler)

    @property
    def feature_size(self):
        """
        Property that returns the total size of the feature vector.

        Returns:
            An integer representing the total number of features.
        """
        return self._feature_size

    def extract(self, state):
        """
        Converts a raw state into a 1D feature vector using RBFs.

        Args:
            state: The observation state (position, velocity).

        Returns:
            The extracted feature vector.
        """
        # Reshape state to 2D array [1, n_dims] for the scaler
        state_2d = state.reshape(1, -1)

        # Normalize the state
        scaled_state = self.scaler.transform(state_2d)

        # Get features from each RBF sampler
        all_features = [
            sampler.transform(scaled_state)[0] for sampler in self.samplers
        ]

        # Concatenate the feature vectors from all samplers into one
        return np.concatenate(all_features)

In [ ]:
# Não altere ou remova esta célula

11. Treine um novo agente no ambiente Mountain Car por 200 episódios utilizando o `RBFFeatureExtractor` com 100 componentes. O agente deve ser treinado com os seguintes parâmetros: taxa de aprendizado $\alpha = 0.01$, fator de desconto $\gamma = 0.99$ e parâmetro de exploração $\varepsilon = 0.1$. Armazene os retornos episódicos, obtidos no método `train`, na variável `rbf_agent`.

In [ ]:
env = gym.make("MountainCar-v0")

# YOUR CODE HERE
# --- Hyperparameters ---
# Define the hyperparameters specified in Task 11
ALPHA = 0.01
GAMMA = 0.99
EPSILON = 0.1
EPISODES = 200
N_COMPONENTS = 100

# 1. Instantiate the RBFFeatureExtractor
# This uses the class implemented in cell 34, configured with
# 100 components as required by the task.
rbf_feat_extractor = RBFFeatureExtractor(
    observation_space=env.observation_space,
    n_components=N_COMPONENTS
)

# 2. Instantiate the SGESAgent
# This uses the agent class from cell 10, passing in the
# RBF extractor and the specified hyperparameters.
agent = SGESAgent(
    observation_space=env.observation_space,
    action_space=env.action_space,
    feature_extractor=rbf_feat_extractor,
    alpha=ALPHA,
    gamma=GAMMA,
    epsilon=EPSILON
)

# 3. Train the agent for 200 episodes
# The train() method executes the Sarsa loop
# and returns the list of total rewards for each episode.
rbf_agent = agent.train(env, EPISODES)

env.close()

mean_return = np.mean(rbf_agent)
print(f"Mean Return: {mean_return:.2f}")

last_mean = np.mean(rbf_agent[-20:])
print(f"Mean Return of the Last 20 Episodes: {last_mean:.2f}")

In [ ]:
assert 'rbf_agent' in vars()

## Análise

12. Utilize a biblioteca matplotlib para construir um gráfico comparativo dos retornos episódicos obtidos pelos agentes utilizando cada um dos construtores de features implementados. Utilize as variáveis `identiy_agent`, `tile_agent` e `rbf_agent` obtidas nos exercícios anteriores. No eixo X, represente os episódios; no eixo Y, o retorno acumulado por episódio. Caso seja necessário, aplique uma média movel para suavizar as curvas e deixar as tendências mais evidentes. Inclua título, legendas e rótulos de eixos apropriados para facilitar a interpretação. Se utilizar algum tipo de suavização, indique claramente no gráfico qual o método aplicado.

In [ ]:
# YOUR CODE HERE
# This cell uses 'matplotlib.pyplot' (as plt) and 'numpy' (as np),
# which were imported in cell 8 of the notebook.

def moving_average(data, window_size):
    """
    Calculates the moving average of a 1D array using convolution.
    Args:
        data (np.ndarray or list): The 1D input data.
        window_size (int): The width of the moving average window.
    Returns:
        np.ndarray: The smoothed data.
    """
    # Use 'valid' mode to ensure the average is only calculated
    # for full windows.
    return np.convolve(data, np.ones(window_size) / window_size, mode='valid')

# Define the window size for smoothing
WINDOW_SIZE = 20

# Apply the moving average to each agent's episodic returns
smooth_identity = moving_average(identity_agent, WINDOW_SIZE)
smooth_tile = moving_average(tile_agent, WINDOW_SIZE)
smooth_rbf = moving_average(rbf_agent, WINDOW_SIZE)

# Create the x-axis (episodes).
# This axis is shifted to align with the 'valid' convolution,
# representing the *end* of each window.
x_axis = np.arange(WINDOW_SIZE - 1, len(identity_agent))

# --- Plotting ---
plt.figure(figsize=(14, 7))

# Plot the smoothed learning curves for each agent
plt.plot(x_axis, smooth_identity, label=f'Identidade (M.M. {WINDOW_SIZE} episódios)')
plt.plot(x_axis, smooth_tile, label=f'Tile Coding (M.M. {WINDOW_SIZE} episódios)')
plt.plot(x_axis, smooth_rbf, label=f'RBF (M.M. {WINDOW_SIZE} episódios)')

# Add plot labels and a title as requested
plt.title('Desempenho dos Extratores de Features no Mountain Car')
plt.xlabel('Episódio')
plt.ylabel('Retorno Acumulado (Média Móvel)')

# Set the y-axis limit to focus on the relevant reward range.
# The worst possible return is -200 (truncation).
plt.ylim([-205, -90])

# Add a legend to identify each line
plt.legend()

# Add a grid for easier reading of the values
plt.grid(True)

# Display the final plot
plt.show()

13. Explique por que a modelagem do construtor de features pode ser crucial para o desempenho de um agente que utiliza aproximação de função.

YOUR ANSWER HERE

A modelagem do construtor de features é crucial porque, em problemas com aproximação de função, o agente não aprende sobre o estado "bruto" (como `(posição, velocidade)`), mas sim sobre o **vetor de features** que representa esse estado[cite: 24, 570].

Com a aproximação linear que usamos, o Q-value é calculado como um produto escalar: $\hat{q}(s,a,\mathbf{w}) = \mathbf{w}^T \mathbf{x}(s,a)$[cite: 24, 120, 570]. Isso significa duas coisas:

1.  **Teto de Desempenho:** A qualidade máxima do agente é limitada pela qualidade das features. Se o vetor $\mathbf{x}(s,a)$ não contiver as informações necessárias para distinguir um estado bom de um estado ruim, o agente **jamais** conseguirá aprender a política ótima, independentemente de quanto ele treine. O construtor `IdentityFeatureExtractor` falha por isso: a função de valor real do Mountain Car é complexa e não-linear (como um "vale" [cite: 553]), e os estados brutos `(posição, velocidade)` não são linearmente separáveis para representá-la.

2.  **Base para Generalização:** Em espaços contínuos, o agente nunca visitará o mesmo estado exato duas vezes. O único jeito de ele aprender é **generalizando** o conhecimento de estados visitados para estados novos, mas similares. As features são o mecanismo para isso[cite: 565].
    * Um bom construtor (como Tile Coding ou RBF) garante que estados próximos no espaço original (ex: `(-0.5, 0.0)` e `(-0.51, 0.0)`) produzam vetores de features parecidos (ex: que compartilhem "tiles" ativos)[cite: 112].
    * Isso faz com que a atualização de valor para um estado "vaze" para seus vizinhos. O agente aprende uma função de valor "suave" e generalizada, em vez de tentar memorizar pontos discretos, o que seria impossível.

Em resumo, o construtor de features é o componente que "traduz" o problema do mundo real para uma forma que o agente linear consegue entender e resolver. Se a tradução for ruim, o agente falha.

14. Quais critérios devem guiar a escolha dos modelos e da quantidade de features a serem utilizadas na construção do espaço de features?

YOUR ANSWER HERE

A escolha do modelo e da quantidade de features deve ser guiada por um balanço entre **poder de discriminação** e **capacidade de generalização**.

1.  **Escolha do Modelo (Ex: Tile Coding vs. RBF):**
    * **Natureza do Problema:** O Tile Coding é excelente para partições rígidas do espaço e generalização local (estados vizinhos ativam features parecidos). RBFs oferecem uma generalização mais suave e global. A escolha depende se o valor da função muda abruptamente ou suavemente no espaço de estados.
    * **Dimensionalidade:** O Tile Coding sofre com a "maldição da dimensionalidade" (features crescem exponencialmente com as dimensões do estado). Métodos como RBFs (especialmente suas aproximações) podem escalar melhor para problemas com mais dimensões.

2.  **Quantidade de Features (Ex: `num_tilings` ou `n_components`):**
    * **Poucas Features (Generalização Excessiva):** Se houver poucas features (ex: poucos *tiles*), o agente não consegue **discriminar** estados diferentes que exigem ações diferentes. Isso é *underfitting*. O agente pode, por exemplo, agrupar o fundo do vale e a subida da montanha no mesmo "bloco", tornando impossível aprender a política correta.
    * **Muitas Features (Memorização):** Se houver features demais (ex: milhares de *tiles*), o agente perde a capacidade de **generalizar**. A experiência de um estado não é "transferida" para seus vizinhos. Isso é *overfitting* (ou memorização). O aprendizado se torna muito lento, pois o agente precisa visitar uma quantidade massiva de estados para aprender os valores de todos os pesos.
    * **O Ponto Ideal:** O objetivo é ter features suficientes para representar as nuances da função de valor (discriminação), mas poucas o suficiente para que o aprendizado de um estado generalize para estados próximos (generalização). Esse balanço é quase sempre encontrado empiricamente, através de ajuste de hiperparâmetros, como visto nos gráficos do livro-texto.